In [1]:
#Bibliotecas
import os
import cv2
import numpy as np
import tensorflow as tf
from google.colab import drive
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator

Utilizando um dataset localizado no google drive é possível treinar o modelo e rotulá-lo

In [2]:
drive.mount('/content/drive')

# Caminho da pasta raiz contendo as subpastas
dataset = "/content/drive/MyDrive/IA_Projects/Face_Recognition/faces_dataset/partial_dataset"

# Rotula dataset
labels = []
for foldername in os.listdir(dataset):
    labels.append(foldername)
labels = np.array(labels)
#print("Número de rótulos encontrados:", len(labels))
#print("Rótulos encontrados:", labels)
#np.save("rotulos.npy", labels)

# Define numero de pessoas no dataset
n_classes = len(labels)

Mounted at /content/drive


Pré processa as imagens, configura-se o shape das imagens  

In [3]:
datagen = ImageDataGenerator(rescale=1./255,
                                   shear_range=0.2, #cisalhamento - trans. geometrica que distorce a imagem e simula angulação em imagens reais 0.2rad
                                   zoom_range=0.2,
                                   horizontal_flip=True) # tecnica de espelhamento, serve para aumentar os dados e treinar o modelo para angulos diferentes

num_samples = len(datagen.flow_from_directory(dataset, batch_size=16, shuffle=False).filenames)

# Ajuste de formato das imagens
train_generator = datagen.flow_from_directory(
    
    dataset,
    target_size=(112, 112),
    batch_size=16,
    shuffle=False,
    class_mode='categorical')

x, y = train_generator.next()

X_train, X_val, y_train, y_val = train_test_split(x, y, test_size=0.2, random_state=42)


Found 2484 images belonging to 579 classes.
Found 2484 images belonging to 579 classes.


In [4]:
model = tf.keras.models.Sequential([
    # primeira camada convolucional
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(112, 112, 3)),
    tf.keras.layers.MaxPooling2D((2, 2)),
    # segunda camada convolucional
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D((2, 2)),
    # terceira camada convolucional
    tf.keras.layers.Conv2D(128, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D((2, 2)),
    # quarta camada convolucional
    tf.keras.layers.Conv2D(128, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(n_classes, activation='softmax')])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])

In [5]:
model.fit(X_train, y_train, batch_size=64, epochs=20, validation_data=(X_val, y_val))


Epoch 1/20
1/1 [==============================] - 2s 2s/step - loss: 6.3877 - categorical_accuracy: 0.0000e+00 - val_loss: 6.0749 - val_categorical_accuracy: 0.2500
Epoch 2/20
1/1 [==============================] - 0s 278ms/step - loss: 5.7826 - categorical_accuracy: 0.4167 - val_loss: 5.1842 - val_categorical_accuracy: 0.2500
Epoch 3/20
1/1 [==============================] - 0s 290ms/step - loss: 4.1898 - categorical_accuracy: 0.4167 - val_loss: 4.9907 - val_categorical_accuracy: 0.2500
Epoch 4/20
1/1 [==============================] - 0s 271ms/step - loss: 2.0171 - categorical_accuracy: 0.4167 - val_loss: 8.4177 - val_categorical_accuracy: 0.2500
Epoch 5/20
1/1 [==============================] - 0s 265ms/step - loss: 1.6844 - categorical_accuracy: 0.6667 - val_loss: 11.2909 - val_categorical_accuracy: 0.2500
Epoch 6/20
1/1 [==============================] - 0s 273ms/step - loss: 1.9439 - categorical_accuracy: 0.4167 - val_loss: 12.4526 - val_categorical_accuracy: 0.2500
Epoch 7/20
1/